## Convert fits file to binary raw file
3D visualization of astronomy data cubes using immersive displays
by Gilles Ferrand, Jayanne English, Pourang Irani

ref: https://arxiv.org/abs/1607.08874

In [21]:
from astropy.io import fits

hdulist = fits.open('l1448_13co.fits')
cube = hdulist[0].data
# And to write the cube to disk as 8-bit integers in a binary file (as expected by my loader on Unity's side):
cube_8bit = ((cube-cube.min())/(cube.max()-cube.min())*255).astype('uint8')


In [26]:
cube_8bit

array([[[44, 44, 51, ..., 40, 69, 79],
        [32, 46, 39, ..., 44, 60, 65],
        [46, 47, 38, ..., 41, 44, 47],
        ..., 
        [50, 31, 38, ..., 41, 31, 23],
        [30, 22, 40, ..., 40, 27, 32],
        [49, 29, 23, ..., 42, 50, 32]],

       [[45, 38, 42, ..., 36, 53, 54],
        [38, 28, 39, ..., 44, 52, 61],
        [42, 33, 41, ..., 54, 69, 76],
        ..., 
        [37, 42, 33, ..., 35, 37, 47],
        [36, 29, 30, ..., 26, 30, 30],
        [50, 32, 43, ..., 35, 39, 54]],

       [[29, 45, 41, ..., 57, 42, 57],
        [38, 35, 28, ..., 54, 74, 72],
        [43, 46, 27, ..., 60, 70, 84],
        ..., 
        [32, 30, 35, ..., 32, 36, 25],
        [38, 28, 24, ..., 40, 43, 47],
        [36, 31, 36, ..., 31, 42, 27]],

       ..., 
       [[66, 50, 46, ..., 36, 29, 47],
        [53, 49, 26, ..., 37, 32, 45],
        [54, 61, 55, ..., 40, 35, 30],
        ..., 
        [43, 64, 59, ..., 38, 32, 43],
        [56, 64, 45, ..., 44, 36, 46],
        [83, 64, 59, ..., 40

In [32]:
# cube size needs to be the same on width, height, and depth - not necessary

# unity has non-power of two

import numpy as np
# npad is a tuple of (n_before, n_after) for each dimension
b = np.pad(cube_8bit, pad_width=((0, 11), (0,23), (0,23)), mode='constant', constant_values=0)
b.transpose().tofile('l1448co.raw')
b.shape

(64, 128, 128)